# Intra-Annual Sea Level Anomalies
Monthly mean sea level anomalies. 

```{caution}
Under construction. Still. Taking my cues from the guys that work on the rail.
```


## Setup

As with previous sections, we first need to import the necessary libraries, establish our input/output directories, and set up some basic plotting rules. We'll do this by running another notebook called "setup" and "plotting functions," and then we'll set our data and output paths. 

In [ ]:
%run ../setup.ipynb
%run ../plotting_functions.ipynb

#import tseries functions
from tseries_functions import process_trend_with_nan

### Import the tide gauge data

In [ ]:
#import rsl_daily
rsl_daily = xr.open_dataset(data_dir/ 'rsl_daily_hawaii.nc')

#make rsl_daily a subset - do not include TGs with more than 25% NaNs
# get percent Nans in rsl_daily['sea_level']

rsl_daily

### check the data and drop all stations with less than 80% coverage

In [ ]:
data_coverage = rsl_daily.rsl_mhhw.count(dim='time')/len(rsl_daily.time)

#drop all locations with data_coverage less than 80%
rsl_subset = rsl_daily.where(data_coverage>0.80,drop=True)

rsl_subset

In [ ]:
# get the record min and max for each TG by month
record_min = rsl_subset.groupby('time.month').min()
record_max = rsl_subset.groupby('time.month').max()

record_min

In [ ]:
# select which station to plot
station_names = rsl_subset['station_name'].values
station_ids = rsl_subset['station_id'].values

#make a dict
station_dict = dict(zip(station_ids, station_names))
station_dict

In [ ]:
# Resample the data to get monthly means and squeeze any singleton dimensions
rsl_monthly = rsl_subset.resample(time='1MS').mean().squeeze()

# Calculate monthly statistics: mean, minimum, and maximum
epochStart = '1993-01-01'
epochEnd = '2024-12-31'
rsl_monthly_mean = rsl_monthly.sel(time=slice(epochStart, epochEnd)).groupby('time.month').mean()
rsl_monthly_min = rsl_monthly.sel(time=slice(epochStart, epochEnd)).groupby('time.month').min()
rsl_monthly_max = rsl_monthly.sel(time=slice(epochStart, epochEnd)).groupby('time.month').max()

station_id = station_ids[0]
station_name = station_dict[station_id]

# Plot the average sea level for each month
ax = rsl_monthly_mean['rsl_mhhw'].sel(station_id=station_id).plot(label='Monthly Average')


# Add shaded area between min and max values for each month
plt.fill_between(rsl_monthly_min.month, 
                 rsl_monthly_min['rsl_mhhw'].sel(station_id=station_id), 
                 rsl_monthly_max['rsl_mhhw'].sel(station_id=station_id), 
                 alpha=0.2, label='Range (Min-Max)')




# Add 2024 monthly values on top of the average
rsl_2024 = rsl_monthly.sel(time=slice('2024-01-01', '2024-12-31')).squeeze()
plt.plot(np.arange(1, 13), rsl_2024['rsl_mhhw'].sel(station_id=station_id).values, 
         color='black', label='2024', marker='o')

# Customize the x-axis to show month names instead of numbers
plt.xticks(np.arange(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Set labels and title
# plt.ylabel('Sea Level Anomaly (m)')
plt.xlabel('')
plt.title(rsl_monthly_mean['station_name'].sel(station_id=station_id).values)

# Adjust x-axis limits and add a grid
plt.xlim(1, 12)
# plt.ylim(-0.45, -0.05)
plt.grid(alpha=0.2, color='lightgray')

# Add a legend at the top in 3 columns
plt.legend(loc='upper center', ncol=3, bbox_to_anchor=(0.5, 1))

# Save the plot to a file
plt.savefig(output_dir / 'SL_MMMA_intra_pacific.png', dpi=300, bbox_inches='tight')

fig = plt.gcf()

glue("SL_MMA_intra",fig,display=False)
glue("station",station_name,display=False)
glue('epochEnd',epochEnd,display=False)
glue('epochStart',epochStart,display=False)


```{glue:figure} SL_MMA_intra
:name: "SL_MMA_intra"

Monthly means at the {glue:text}`station` tide gauge for {glue:text}`epochStart` to {glue:text}`epochEnd`. Note that the sea level is plotted in units of m, relative to MHHW. Note also that this differs from the plot available at https://uhslc.soest.hawaii.edu/stations/?stn=060#climatology, because we are using different periods of record. 
```

## Absolute Value: satellite (trend removed)

In [ ]:
# open the cmems data
cmems = xr.open_dataset(data_dir / 'cmems_L4_SSH_0.125deg_1993_2025hawaii.nc')

#reduce to smaller area
cmems = cmems.sel(longitude=slice(180, 210), latitude=slice(15, 32))

### Plot an area-wide map, with stations
First we'll need to make sure that our data is detrended properly in order to look at monthly anomalies.

In [ ]:
# remove long term trend from rsl using polynomial fit
trend_mag_cmems, trend_line_cmems, trend_rate_cmems = process_trend_with_nan(cmems.sla)

cmems_detrended = cmems.sla - trend_line_cmems

# make a cmems monthly mean for plotting
cmems_monthly = cmems_detrended.resample(time='1MS').mean().squeeze()

# get the mean for each month of the year (climatology)
cmems_climatology = cmems_monthly.groupby('time.month').mean(dim='time')

# get the anomaly by month (subtract the climatology from the monthly means))
cmems_mmsla = cmems_monthly.groupby('time.month') - cmems_climatology


Get the absolute sea level climatology at the tide gauge locations.

In [ ]:
cmems_climatology_tg = cmems_climatology.interp(longitude=rsl_monthly.longitude, latitude=rsl_monthly.latitude, method='nearest')


In [ ]:
cmems_climatology_tg

In [ ]:
#compare cmems_climatology_tg with rsl_monthly_mean
fig, axes = plt.subplots(nrows=7, ncols=1, figsize=(5, 10), sharex=True, sharey=True)
axes = axes.flatten()
# iterate over each axis and plot the monthly mean
for i in range(len(station_ids)):
    ax = axes[i]
    (rsl_monthly_mean['rsl_anomaly'].sel(station_id=station_ids[i])- rsl_monthly_mean['rsl_anomaly'].sel(station_id=station_ids[i]).mean()).plot(ax = ax, x='month',label='RSL Climatology')
    # Add cmems climatology at the station location
    cmems_climatology_tg.sel(station_id=station_ids[i]).plot.line(ax=ax, x='month', label='CMEMS Climatology', color='orange', linestyle='--')

    ax.set_ylim([-0.07,0.07])
    ax.set_xlim([1,12])

    ax.text(0.03,0.8,rsl_monthly['station_name'].values[i], fontsize=10,transform=ax.transAxes)
    ax.set_ylabel(None)
    ax.set_title(None)
    ax.set_xlabel(None)

    if i == 0:
        ax.legend(loc='lower center', ncol=3)   


# add a common x and y label
axes[-1].set_xlabel('Month')
fig.text(0.02, 0.5, 'Monthly Mean Sea Level Anomaly [m]', ha='center', va='center', rotation='vertical')

In [ ]:
# plot the monthly_anomalies*100 for centimeters
cmems_cm = cmems_climatology * 100



# convert cmems_cm.month to datetime object
months = pd.to_datetime(cmems_cm.month.values, format='%m').strftime('%b')


# Plot the data with modified titles
mma = cmems_cm.plot(transform=ccrs.PlateCarree(), col='month', col_wrap=3,
                        aspect=cmems_cm.sizes['longitude'] / cmems_cm.sizes['latitude'],
                        subplot_kws={'projection': ccrs.PlateCarree(central_longitude=180)},)

# Add coastlines and land features to each subplot
for i,ax in enumerate(mma.axs.flat):
    ax.coastlines(linewidth=0.5)
    ax.set_title(months[i])
    ax.set_extent([180, 207,  15, 30]) 

# change the colorbar label
mma.cbar.set_label('Monthly Mean Sea Level Anomaly (cm)')  

#assign mma to a figure
fig = plt.gcf()

# change figure size
fig.set_size_inches(8, 6)

#save the figure
fig.savefig(output_dir / 'SL_anomaly_map_pacific.png', dpi=300)

glue("mma_asl_fig", fig, display=False)



```{glue:figure} mma_asl_fig
:name: "mma-asl-fig"
Map of absolute monthly mean sea level anomalies from altimetry for the full record from {glue:text}`station_group`, {glue:text}`country` station from {glue:text}`startDateTime` to {glue:text}`endDateTime`. This plot is our monthly climatology - sea levels in Hawaii are on average lower in the spring (March-May) and higher in the fall (September-November). 
```


In [ ]:
# we can also plot any given year against the climatology
year = '2023'
yearStart = f'{year}-01-01'
yearEnd = f'{year}-12-31'

cmems_Year = cmems_mmsla.sel(time=slice(yearStart, yearEnd)).squeeze()
# get monthly means
cmems_Year_monthly = cmems_Year.groupby('time.month').mean(dim='time')

cmems_Year_monthly_anomaly = cmems_Year_monthly - cmems_climatology
# plot the monthly_anomalies*100 for centimeters
cmems_Year_cm = cmems_Year_monthly_anomaly * 100


In [ ]:
# Plot the data with modified titles
mma = cmems_Year_cm.plot(transform=ccrs.PlateCarree(), col='month', col_wrap=3,
                        aspect=cmems_Year_cm.sizes['longitude'] / cmems_Year_cm.sizes['latitude'],
                        subplot_kws={'projection': ccrs.PlateCarree(central_longitude=180)},)

# Add coastlines and land features to each subplot
for i,ax in enumerate(mma.axs.flat):
    ax.coastlines(linewidth=0.5)
    ax.set_title(months[i])
    ax.set_extent([180, 207,  15, 30]) 

# change the colorbar label
mma.cbar.set_label(f'{year} Monthly Mean Sea Level Anomaly (cm)')  

#assign mma to a figure
fig = plt.gcf()

# change figure size
fig.set_size_inches(8, 6)

Let's try that in a movie.

In [ ]:
import imageio.v2 as imageio

frames = []
for i, month in enumerate(months):
    fig, ax = plt.subplots(subplot_kw={'projection': ccrs.PlateCarree(central_longitude=180)}, figsize=(6,3))
    cmems_Year_cm.sel(month=i+1).plot(ax=ax, transform=ccrs.PlateCarree(),vmin=-25, vmax=25, cmap='RdBu_r', cbar_kwargs={'label': f'{year} Monthly Mean \nSea Level Anomaly (cm)'})
    ax.coastlines(linewidth=0.5)
    ax.set_title(month)
    ax.set_extent([180, 207, 15, 30])
    # Save each month as a PNG
    fname = output_dir / f"SL_anomaly_map_pacific_{month}.png"
    fig.savefig(fname, dpi=150)
    frames.append(str(fname))
    plt.close(fig)

# Create GIF from saved PNGs
images = [imageio.imread(frame) for frame in frames]
gif_path = output_dir / "SL_anomaly_map_pacific.gif"
imageio.mimsave(gif_path, images, fps=5, loop=0)


![It's Moving!](https://uhslc.soest.hawaii.edu/jfiedler/SeaLevelIndicators/plotly/SL_anomaly_map_pacific.gif)

## Relative Value: tide station (trend removed)
### area-wide map with stations
### station plots: time series  
These should have ONI/ENSO phase included on them

In [ ]:
rsl_monthly

In [ ]:
# remove long-term trend from rsl_monthly
trend_mag_rsl, trend_line_rsl, trend_rate_rsl = process_trend_with_nan(rsl_monthly.rsl_anomaly)

rsl_monthly_detrended = rsl_monthly.rsl_anomaly - trend_line_rsl

# add detrended rsl_monthly to rsl_monthly
rsl_monthly['sea_level_detrended'] = rsl_monthly_detrended
rsl_monthly['sea_level_detrended'].attrs = {'long_name': 'sea level anomaly', 'units': 'm'}

In [ ]:
# remove annual cycle from rsl_monthly
rsl_monthly_detrended = rsl_monthly_detrended.groupby('time.month') - rsl_monthly_detrended.groupby('time.month').mean(dim='time')

# add to rsl_monthly
rsl_monthly['sea_level_detrended_anom'] = rsl_monthly_detrended
rsl_monthly['sea_level_detrended_anom'].attrs = {'long_name': 'monthly mean sea level anomaly', 'units': 'm'}

## Make a plot

In [ ]:
rsl_monthly.sel(station_id=station_id)

In [ ]:
# get axis for the plots
fig,ax = plt.subplots(1,1,figsize=(10,4), sharex=True, sharey=True)

# add ENSO to the plots
# import oni.csv
oni = pd.read_csv(data_dir/'climate_indices'/'oni_bool.csv',header=0)
# change oni index to Date column
oni.index = pd.to_datetime(oni['Date'] )



rsl_monthly['sea_level_detrended_anom'].sel(station_id=station_id).plot.line(ax=ax, x='time', label='monthly sea level anomaly [m]', color='black')
# Shading El Niño events
ax.fill_between(oni.index, oni['ONI'].min(), oni['ONI'].max(), 
                where=oni['El Nino'] == 1, color='red', alpha=0.2, label='El Niño')
# Shading La Niña events
ax.fill_between(oni.index, oni['ONI'].min(), oni['ONI'].max(), 
                where=oni['La Nina'] == 1, color='blue', alpha=0.2, label='La Niña')

ax.set_xlim([dt.datetime(1993,1,1),dt.datetime(2024,1,1)])
ax.set_ylim([-0.3,0.3])
ax.set_title(rsl_monthly['station_name'].sel(station_id=station_id).values)
ax.set_ylabel('Monthly Mean Sea Level Anomaly [m]')

 #put legend on the side
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#save the figure
# fig.savefig(output_dir / 'SL_anomaly_timseries_pacific.png', dpi=300)
# glue("SL_anomaly_timeseries_honolulu_fig", fig, display=False)

In [ ]:

# get axis for the plots
fig,axes = plt.subplots(4,2,figsize=(15,10), sharex=True, sharey=True)
axes = axes.flatten()

# add ENSO to the plots
# import oni.csv
oni = pd.read_csv(data_dir/'climate_indices'/'oni_bool.csv',header=0)
# change oni index to Date column
oni.index = pd.to_datetime(oni['Date'] )


# iterate over each axis and plot the monthly mean
for i in range(len(station_ids)):
    ax = axes[i]
    rsl_monthly['sea_level_detrended_anom'].sel(station_id=station_ids[i]).plot.line(ax=ax, x='time', label='monthly sea level anomaly [m]', color='black',linewidth=1)

    # Shading El Niño events
    ax.fill_between(oni.index, oni['ONI'].min(), oni['ONI'].max(), 
                    where=oni['El Nino'] == 1, color='red', alpha=0.2, label='El Niño')

    # Shading La Niña events
    ax.fill_between(oni.index, oni['ONI'].min(), oni['ONI'].max(), 
                    where=oni['La Nina'] == 1, color='blue', alpha=0.2, label='La Niña')
    
    ax.set_xlim([dt.datetime(1993,1,1),dt.datetime(2024,1,1)])
    ax.set_ylim([-0.3,0.3])

    ax.set_title(rsl_monthly['station_name'].values[i] + f' ({station_ids[i]})')
    ax.set_ylabel('Monthly \nSea Level Anomaly [m]')

    if i < 6:
        ax.set_xlabel('')

# remove empty subplots
for i in range(len(station_ids), len(axes)):
    fig.delaxes(axes[i])

#put legend on the side
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

#save the figure
fig.savefig(output_dir / 'SL_anomaly_timeseries_stations.png', dpi=300)

glue("SL_anomaly_timeseries_stations_fig", fig, display=False)


In [ ]:
# make the same plot but only on one axis
fig,ax = plt.subplots()


rsl_monthly['sea_level_detrended_anom'].plot.line(x='time', label=rsl_monthly['station_name'].values)

# add ONI shading
# Shading El Niño events
ax.fill_between(oni.index, oni['ONI'].min(), oni['ONI'].max(), 
                where=oni['El Nino'] == 1, color='red', alpha=0.2, label='El Niño')

# Shading La Niña events
ax.fill_between(oni.index, oni['ONI'].min(), oni['ONI'].max(), 
                where=oni['La Nina'] == 1, color='blue', alpha=0.2, label='La Niña')

ax.set_xlim([dt.datetime(1993,1,1),dt.datetime(2024,1,1)])
ax.set_ylim([-0.3,0.3])

#put the legend outside the plot
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

ax.set_title('Monthly Mean Sea Level Anomalies (1993-2023)')

# add mean sea level anomaly to the plot
rsl_monthly['sea_level_detrended_anom'].mean(dim='station_id').plot.line(x='time', label='mean sea level anomaly [m]', color='black')

#update the legend
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# change ylabel
ax.set_ylabel('Monthly Mean Sea Level Anomaly [m]')

#save the figure
fig.savefig(output_dir / 'SL_anomaly_timeseries.png', dpi=300)

glue("SL_anomaly_timeseries_fig", fig, display=False)


```{glue:figure} SL_anomaly_timeseries_fig
:name: "SL_anomaly_timeseries"
Plot of absolute monthly mean sea level anomalies from tide gauges for the full record from the Hawaiian Island region stations, station from {glue:text}`startDateTime` to {glue:text}`endDateTime`.
```
